In [1]:
import os
import sys
import time
import joblib
import numpy as np
import cv2
from tqdm import tqdm
from skimage.feature import local_binary_pattern, hog
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

def is_dfire_image_fire(annotation_path, fire_class_ids):
    if not os.path.exists(annotation_path): return False
    try:
        with open(annotation_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if parts and len(parts) > 0:
                    if parts[0].isdigit():
                        class_id = int(parts[0])
                        if class_id in fire_class_ids:
                            return True
    except Exception as e: pass
    return False

def extract_color_histograms(img_processed, color_space, bins):
    histograms = []
    ranges = {
        'hsv': {'float': ([0, 1], [0, 1], [0, 1]), 'uint8': ([0, 180], [0, 256], [0, 256])},
        'ycbcr': {'float': ([0, 1], [-0.5, 0.5], [-0.5, 0.5]), 'uint8': ([0, 256], [0, 256], [0, 256])}
    }
    channel_indices = {'hsv': [0, 1], 'ycbcr': [1, 2]}
    dtype_key = 'float' if img_processed.dtype in [np.float32, np.float64] else 'uint8'
    if color_space in ranges and color_space in channel_indices:
        for i in channel_indices[color_space]:
            if img_processed.dtype != np.float32 and img_processed.dtype != np.uint8:
                 img_processed = img_processed.astype(np.float32 if normalize_pixels else np.uint8)
                 dtype_key = 'float' if img_processed.dtype in [np.float32, np.float64] else 'uint8'
            current_range = ranges[color_space][dtype_key][i]
            try:
                hist = cv2.calcHist([img_processed], [i], None, [bins], current_range)
                histograms.append(hist.flatten())
            except Exception as e: pass
    if histograms:
        return np.concatenate(histograms)
    else:
        return np.array([]) 

def extract_lbp_features(img_gray, radius, n_points, method):
    if img_gray is None or img_gray.size == 0:
         return np.array([])
    if img_gray.dtype != np.uint8 and img_gray.dtype != np.float64:
         img_gray = img_gray.astype(np.float64)
    if n_points is None:
        n_points = 8 * radius

    try:
        lbp_image = local_binary_pattern(img_gray, n_points, radius, method=method)
        if method == 'uniform' or method == 'nri_uniform':
            n_bins = int(n_points + 2) 
            hist_range = (0, n_bins)
        elif method == 'ror':
            n_bins = int(n_points / radius + 2) 
            hist_range = (0, n_bins)
        else: 
            n_bins = int(2**n_points)
            hist_range = (0, n_bins)

        lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=n_bins, range=hist_range)
        lbp_hist = lbp_hist.astype(np.float32) 
        if lbp_hist.sum() > 0:
            lbp_hist /= lbp_hist.sum()
        return lbp_hist.flatten()
    except Exception as e: return np.array([]) 

def extract_hog_features(img_gray, orientations, pixels_per_cell, cells_per_block, block_norm):
    if img_gray is None or img_gray.size == 0: return np.array([])
    if img_gray.dtype != np.uint8 and img_gray.dtype != np.float64:
         img_gray = img_gray.astype(np.float64)
    img_h, img_w = img_gray.shape
    cell_h, cell_w = pixels_per_cell
    block_h, block_w = cells_per_block
    min_img_h = cell_h * block_h
    min_img_w = cell_w * block_w
    if img_h < min_img_h or img_w < min_img_w:
        return np.array([]) 

    try:
        hog_features = hog(img_gray, orientations=orientations,
                           pixels_per_cell=pixels_per_cell,
                           cells_per_block=cells_per_block,
                           block_norm=block_norm,
                           visualize=False, feature_vector=True)
        return hog_features.flatten().astype(np.float32) 
    except Exception as e:
        return np.array([]) 


def combine_features(img_dict, feature_params):
    all_features = []
    if 'hsv' in img_dict and img_dict['hsv'] is not None:
        hsv_hist = extract_color_histograms(img_dict['hsv'], 'hsv', bins=feature_params.get('hist_bins', 100))
        if hsv_hist.size > 0:
            all_features.append(hsv_hist)
    if 'ycbcr' in img_dict and img_dict['ycbcr'] is not None:
        ycbcr_hist = extract_color_histograms(img_dict['ycbcr'], 'ycbcr', bins=feature_params.get('hist_bins', 100))
        if ycbcr_hist.size > 0:
            all_features.append(ycbcr_hist)
    
    if 'gray' in img_dict and img_dict['gray'] is not None:
        img_gray_processed = img_dict['gray'] 
        lbp_features = extract_lbp_features(img_gray_processed,
                                            radius=feature_params.get('lbp_radius', 3),
                                            n_points=feature_params.get('lbp_n_points', None),
                                            method=feature_params.get('lbp_method', 'uniform'))
        if lbp_features.size > 0:
            all_features.append(lbp_features)

        
        hog_features = extract_hog_features(img_gray_processed,
                                           orientations=feature_params.get('hog_orientations', 9),
                                           pixels_per_cell=feature_params.get('hog_pixels_per_cell', (8, 8)),
                                           cells_per_block=feature_params.get('hog_cells_per_block', (2, 2)),
                                           block_norm=feature_params.get('hog_block_norm', 'L2-Hys'))
        if hog_features.size > 0:
            all_features.append(hog_features)

    if all_features:
        combined_vector = np.concatenate(all_features)
        return combined_vector.astype(np.float32) 
    else:
        return np.array([]) 


def get_config(dataset_choice):
    config = {}
    if dataset_choice == 'kaggle':
        config['dataset_choice'] = 'kaggle'
        config['data_root'] = os.path.join('..', 'data_subsets', 'fire_dataset')
        config['target_img_size'] = (128, 128)
        config['color_spaces_to_load'] = ['bgr', 'hsv', 'ycbcr']
        config['normalize_pixels'] = 1
        config['fire_class_ids'] = None 
    elif dataset_choice == 'dfire':
        config['dataset_choice'] = 'dfire'
        config['dfire_root'] = os.path.join('..', 'data_subsets', 'D-Fire')
        config['split_name'] = "test"
        config['data_root'] = os.path.join(config['dfire_root'], config['split_name'])
        config['target_img_size'] = (128, 128)
        config['color_spaces_to_load'] = ['bgr', 'hsv', 'ycbcr']
        config['normalize_pixels'] = 1
        config['fire_class_ids'] = [0, 1] 
    else:
        raise ValueError(f"Unknown dataset choice: {dataset_choice}. Choose 'kaggle' or 'dfire'.")

    print(f"Using dataset: {config['dataset_choice']} ({config.get('split_name', 'N/A')} split)")
    print(f"Data root: {config.get('data_root')}")
    print(f"Target image size: {config['target_img_size']}")
    print(f"Color spaces loaded: {config['color_spaces_to_load']}")
    print(f"Normalize pixels: {bool(config['normalize_pixels'])}\n")
    if dataset_choice == 'dfire':
         print(f"D-Fire Fire Class IDs considered fire: {config['fire_class_ids']}\n")
    return config

def get_feature_params():
    feature_params = {
        'hist_bins': 100,
        'lbp_radius': 3,
        'lbp_n_points': None, 
        'lbp_method': 'uniform',
        'hog_orientations': 9,
        'hog_pixels_per_cell': (8, 8),
        'hog_cells_per_block': (2, 2),
        'hog_block_norm': 'L2-Hys'
    }
    return feature_params

def load_test_data_and_extract_raw_features(config, feature_params):
    dataset_choice = config.get('dataset_choice')
    data_root = config.get('data_root')
    target_size = config.get('target_img_size')
    color_spaces_to_load = config.get('color_spaces_to_load')
    normalize_pixels = config.get('normalize_pixels')
    fire_class_ids = config.get('fire_class_ids')

    if not data_root or not target_size:
        print("Error: Data root or target size not specified in config.")
        return np.array([]), np.array([])
    
    image_label_pairs = []
    img_extensions = ['.jpg', '.jpeg', '.png']
    if dataset_choice == 'dfire':
        annotation_extension = '.txt'
        images_dir = os.path.join(data_root, 'images')
        labels_dir = os.path.join(data_root, 'labels')

        if not os.path.isdir(images_dir) or not os.path.isdir(labels_dir):
            print(f"Error: Images or Labels directory not found in {data_root}")
            return np.array([]), np.array([])
        all_image_files = [f for f in os.listdir(images_dir) if os.path.splitext(f)[1].lower() in img_extensions]
        if not all_image_files:
            print(f"No image files found in {images_dir}")
            return np.array([]), np.array([])
        for filename in tqdm(all_image_files, desc="Determining Labels", leave=False):
            image_name_without_ext = os.path.splitext(filename)[0]
            annotation_path = os.path.join(labels_dir, image_name_without_ext + annotation_extension)
            label = 1 if is_dfire_image_fire(annotation_path, fire_class_ids) else 0
            image_label_pairs.append((os.path.join(images_dir, filename), label))

    elif dataset_choice == 'kaggle':
        fire_dir = os.path.join(data_root, 'fire_images')
        non_fire_dir = os.path.join(data_root, 'non_fire_images')
        if not os.path.isdir(fire_dir) or not os.path.isdir(non_fire_dir):
            print(f"Error: 'fire_images' or 'non_fire_images' directory not found in {data_root}")
            return np.array([]), np.array([])
        print("\nDetermining Test Set Labels (Kaggle structure)...")
        fire_image_files = [os.path.join(fire_dir, f) for f in os.listdir(fire_dir) if os.path.splitext(f)[1].lower() in img_extensions]
        for img_path in tqdm(fire_image_files, desc="Processing Fire Images", leave=False):
            image_label_pairs.append((img_path, 1))
        non_fire_image_files = [os.path.join(non_fire_dir, f) for f in os.listdir(non_fire_dir) if os.path.splitext(f)[1].lower() in img_extensions]
        for img_path in tqdm(non_fire_image_files, desc="Processing Non-Fire Images", leave=False):
            image_label_pairs.append((img_path, 0))
        print("Test Set Label determination complete.")
    if not image_label_pairs:
        print("No test images with labels found.")
        return np.array([]), np.array([])
    all_features_list = []
    all_labels_list = []
    total_images_processed = 0
    total_images_skipped_reading = 0
    total_images_skipped_feature = 0
    for image_path, label in tqdm(image_label_pairs, desc="Extracting Raw Features", leave=False):
        img_bgr = cv2.imread(image_path)
        if img_bgr is None:
            total_images_skipped_reading += 1
            continue
        try:
            img_resized = cv2.resize(img_bgr, target_size, interpolation=cv2.INTER_LINEAR)
            img_dict_single = {} 
            img_processed_bgr = None
            if normalize_pixels:
                img_processed_bgr = img_resized.astype(np.float32) / 255.0
                img_resized_uint8 = img_resized.astype(np.uint8)
                img_gray = cv2.cvtColor(img_resized_uint8, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
                if 'hsv' in color_spaces_to_load:
                    img_dict_single['hsv'] = cv2.cvtColor(img_resized_uint8, cv2.COLOR_BGR2HSV).astype(np.float32) / np.array([180, 255, 255], dtype=np.float32) 
                if 'ycbcr' in color_spaces_to_load:
                    img_dict_single['ycbcr'] = cv2.cvtColor(img_resized_uint8, cv2.COLOR_BGR2YCrCb).astype(np.float32) / 255.0 
            else: 
                img_processed_bgr = img_resized.astype(np.uint8)
                img_gray = cv2.cvtColor(img_processed_bgr, cv2.COLOR_BGR2GRAY)
                if 'hsv' in color_spaces_to_load:
                    img_dict_single['hsv'] = cv2.cvtColor(img_processed_bgr, cv2.COLOR_BGR2HSV)
                if 'ycbcr' in color_spaces_to_load:
                    img_dict_single['ycbcr'] = cv2.cvtColor(img_processed_bgr, cv2.COLOR_BGR2YCrCb)
            img_dict_single['gray'] = img_gray
            if 'bgr' in color_spaces_to_load:
                img_dict_single['bgr'] = img_processed_bgr
            features_single = combine_features(img_dict_single, feature_params)
            if features_single.size > 0:
                all_features_list.append(features_single)
                all_labels_list.append(label)
                total_images_processed += 1
            else:
                total_images_skipped_feature += 1 
        except Exception as e:
            total_images_skipped_feature += 1 
    print("Raw Feature extraction complete.")
    print(f"Total images initially found: {len(image_label_pairs)}")
    print(f"Images skipped (read error): {total_images_skipped_reading}")
    print(f"Images skipped (feature error): {total_images_skipped_feature}")
    print(f"Images successfully processed for features: {total_images_processed}")
    if not all_features_list:
        print("No raw features extracted from any test image.")
        return np.array([]), np.array([])
    features_array = np.array(all_features_list, dtype=np.float32) 
    labels_array = np.array(all_labels_list, dtype=np.int32)
    print(f"Raw features array shape: {features_array.shape}")
    print(f"Labels array shape: {labels_array.shape}")
    return features_array, labels_array

def preprocess_features_for_model(X_raw, scaler, selector=None):
    if X_raw is None or X_raw.shape[0] == 0:
        print("Preprocessing skipped: raw features are empty.")
        return np.array([])
    if scaler is None:
        print("Error: Scaler not loaded. Cannot preprocess features.")
        return np.array([])
    X_scaled = scaler.transform(X_raw)
    print(f"Features after scaling: {X_scaled.shape}")
    if selector:
        X_selected = selector.transform(X_scaled)
        print(f"Features after selection: {X_selected.shape}")
        return X_selected
    else:
        print("No feature selection applied.")
        return X_scaled

def evaluate_model_on_test(model, X_test, y_test, model_name, feature_set_name):
    if model is None or X_test is None or y_test is None or X_test.shape[0] == 0:
        print(f"{model_name} evaluation skipped on {feature_set_name}: model not loaded or test data is empty.")
        return
    print(f"\n--- Evaluating {model_name} on D-Fire Test Set ({feature_set_name}) ---")
    start_time = time.time()
    y_pred = model.predict(X_test)
    end_time = time.time()
    print(f"Prediction duration: {end_time - start_time:.4f} seconds")
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"\nConfusion Matrix ({model_name} on {feature_set_name}):")
    print(conf_matrix)
    print("-" * 50)

MODEL_SAVE_DIR = os.path.join('..', 'models')
SCALER_FILENAME = 'scaler_initial.pkl'
SCALER_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, SCALER_FILENAME)

print("--- Starting Model Testing on D-Fire Test Set ---")
try:
    test_config = get_config('dfire')
    test_config['split_name'] = 'test'
    test_config['data_root'] = os.path.join(test_config['dfire_root'], test_config['split_name'])
    print("\nUsing Test Configuration:")
    print(f"  Data Root: {test_config['data_root']}")
    print(f"  Split: {test_config['split_name']}\n")
    feature_params = get_feature_params()

except ValueError as e:
    print(f"Configuration Error: {e}")
    sys.exit(1)
X_test_raw, y_test = load_test_data_and_extract_raw_features(test_config, feature_params)
expected_raw_feature_count = X_test_raw.shape[1]
if X_test_raw.shape[0] == 0:
    print("\nNo test data loaded or features extracted. Exiting.")
    sys.exit(1) 

initial_scaler = None
if os.path.exists(SCALER_SAVE_PATH):
    try:
        initial_scaler = joblib.load(SCALER_SAVE_PATH)
        print(f"\nLoaded initial scaler from: {SCALER_SAVE_PATH}")
    except Exception as e:
        print(f"\nError loading initial scaler {SCALER_SAVE_PATH}: {e}")
else:
    print(f"\nError: Initial scaler not found at {SCALER_SAVE_PATH}. Cannot preprocess features. Exiting.")
    sys.exit(1)


print("\nSearching for saved models in:", MODEL_SAVE_DIR)
saved_models_files = [f for f in os.listdir(MODEL_SAVE_DIR) if f.endswith('.pkl') and '_best_model_' in f]
if not saved_models_files:
    print("No saved models found in", MODEL_SAVE_DIR)
else:
    print(f"Found {len(saved_models_files)} potential saved models.")
    for model_filename in saved_models_files:
        print(f"\nProcessing saved model file: {model_filename}")
        model_path = os.path.join(MODEL_SAVE_DIR, model_filename)
        model = None
        feature_selector = None
        selector_path = None
        model_name = "Unknown"
        feature_set_name = "Unknown"
        try:
            model = joblib.load(model_path)
            parts = model_filename.replace('.pkl', '').split('_best_model_')
            if len(parts) == 2:
                prefix = parts[0] 
                feature_set_name = parts[1] 
                prefix_lower = prefix.lower()
                if 'svm' in prefix_lower:
                    model_name = 'SVM'
                elif 'lightgbm' in prefix_lower:
                    model_name = 'LightGBM'
                elif 'mlp' in prefix_lower:
                    model_name = 'MLP'
                print(f"   Identified Model Type: {model_name}")
                print(f"   Identified Feature Set: {feature_set_name}")
                if hasattr(model, 'n_features_in_'):
                    expected_model_features = model.n_features_in_
                    print(f"   Model expects {expected_model_features} features.")
                    if feature_set_name != 'Scaled_All':
                        selector_filename = f'selector_{feature_set_name}.pkl'
                        selector_path = os.path.join(MODEL_SAVE_DIR, selector_filename)
                        if os.path.exists(selector_path):
                            try:
                                feature_selector = joblib.load(selector_path)
                                print(f"   Loaded feature selection transformer from: {selector_path}")             
                            except Exception as e:
                                print(f"   Error loading selector {selector_path}: {e}. Cannot test this model properly.")
                                feature_selector = None      
                                model = None
                        else:
                            print(f"   Warning: Selector file not found at {selector_path} for feature set {feature_set_name}. Cannot test this model properly.")
                            model = None
                            
                    if model is not None: 
                        X_test_processed = preprocess_features_for_model(X_test_raw, initial_scaler, feature_selector)
                        if X_test_processed.shape[0] > 0 and X_test_processed.shape[1] == expected_model_features:     
                            evaluate_model_on_test(model, X_test_processed, y_test, model_name, feature_set_name)
                        elif X_test_processed.shape[0] == 0:
                            print(f"   Skipping evaluation for {model_name} on {feature_set_name}: Processed test features are empty.")
                        else:     
                            print(f"   Skipping evaluation for {model_name} on {feature_set_name} due to dimension mismatch.")
                            print(f"   Processed test features have {X_test_processed.shape[1]} features, but model expects {expected_model_features}.")

                else:
                    print(f"   Warning: Model {model_filename} does not have 'n_features_in_' attribute. Cannot verify dimension match.")
                    print("   Attempting preprocessing and evaluation anyway.")
                    X_test_processed = preprocess_features_for_model(X_test_raw, initial_scaler, feature_selector)
                    if X_test_processed.shape[0] > 0:
                        evaluate_model_on_test(model, X_test_processed, y_test, model_name, feature_set_name)
                    else:
                        print(f"   Skipping evaluation for {model_name} on {feature_set_name}: Processed test features are empty.")
            else:
                print(f"   Warning: Filename format not recognized: {model_filename}. Skipping.")
        except Exception as e:
            print(f"   Error loading or processing model {model_filename}: {e}. Skipping evaluation.")
            continue
            
print("\n--- Model Testing Complete ---")

--- Starting Model Testing on D-Fire Test Set ---
Using dataset: dfire (test split)
Data root: ..\data_subsets\D-Fire\test
Target image size: (128, 128)
Color spaces loaded: ['bgr', 'hsv', 'ycbcr']
Normalize pixels: True

D-Fire Fire Class IDs considered fire: [0, 1]


Using Test Configuration:
  Data Root: ..\data_subsets\D-Fire\test
  Split: test



Raw Feature extraction complete.
Total images initially found: 1635
Images skipped (read error): 0
Images skipped (feature error): 0
Images successfully processed for features: 1635
Raw features array shape: (1635, 8526)
Labels array shape: (1635,)

Loaded initial scaler from: ..\models\scaler_initial.pkl

Searching for saved models in: ..\models
Found 10 potential saved models.

Processing saved model file: kagglelightgbm_best_model_Scaled_Corr50%.pkl
   Identified Model Type: LightGBM
   Identified Feature Set: Scaled_Corr50%
   Model expects 4263 features.
   Loaded feature selection transformer from: ..\models\selector_Scaled_Corr50%.pkl
Features after scaling: (1635, 8526)
Features after selection: (1635, 4263)

--- Evaluating LightGBM on D-Fire Test Set (Scaled_Corr50%) ---
Prediction duration: 3.1215 seconds
Accuracy: 0.4985
Precision: 0.4993
Recall (Sensitivity): 0.9071
F1 Score: 0.6441

Confusion Matrix (LightGBM on Scaled_Corr50%):
[[ 73 744]
 [ 76 742]]
---------------------